In [0]:
catalog_name = dbutils.widgets.text("catalog_name", "")
catalog_name = dbutils.widgets.get("catalog_name")
bronze_schema = dbutils.widgets.text("bronze_schema", "")
bronze_schema = dbutils.widgets.get("bronze_schema")
silver_schema = dbutils.widgets.text("silver_schema", "")
silver_schema = dbutils.widgets.get("silver_schema")
gold_schema = dbutils.widgets.text("gold_schema", "")
gold_schema = dbutils.widgets.get("gold_schema")
input_table_g_ventas = dbutils.widgets.text("input_table_g_ventas", "")
input_table_g_ventas = dbutils.widgets.get("input_table_g_ventas")
output_table_g_tiempo = dbutils.widgets.text("output_table_g_tiempo", "")
output_table_g_tiempo = dbutils.widgets.get("output_table_g_tiempo")

In [0]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import col, trim, initcap, when, lit, year, month, dayofmonth, concat, concat_ws
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException
import time

In [0]:
schema_auditoria ="auditoria"

In [0]:
dim_tiempo = (
    spark.table(f"{catalog_name}.{silver_schema}.{input_table_g_ventas}")
    .select(
        col("fecha_venta").alias("fecha")
    ).dropna().distinct()
    .withColumn("anio", year(col("fecha")))
    .withColumn("mes", month(col("fecha")))
    .withColumn("dia", dayofmonth(col("fecha")))
    .withColumn("semestre", concat_ws("-",col("anio"), when(col("mes") <=7, lit("01")).otherwise(lit("02"))))
    .withColumn("id_tiempo", (col("anio")*10000 + col("mes")*100 + col("dia")).cast("int"))
)


In [0]:
display(dim_tiempo)

In [0]:
dim_tiempo.write.mode("overwrite").format("delta").saveAsTable(f"{catalog_name}.{gold_schema}.{output_table_g_tiempo}")

In [0]:
spark.table(f"{catalog_name}.{gold_schema}.{output_table_g_tiempo}").show(10)

In [0]:

sql_command = f"""
  OPTIMIZE {catalog_name}.{gold_schema}.{output_table_g_tiempo}
  ZORDER BY (id_tiempo)
"""

In [0]:
spark.sql(sql_command)

In [0]:
spark.sql(f"VACUUM {catalog_name}.{gold_schema}.{output_table_g_tiempo}")